In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import numpy as np

## 依照指示取出模型特定層的資訊

In [2]:
# Define model
class TheModelClass(nn.Module):
    def __init__(self):
        super(TheModelClass, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16* 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model
model = TheModelClass()


### 取出 self.pool層兩次的輸出，包含：
* x = self.pool(F.relu(self.conv1(x)))
* x = self.pool(F.relu(self.conv2(x)))

In [3]:
model.children

<bound method Module.children of TheModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)>

In [4]:
outputs= []
def layer1_hook(module, input_, output):
    outputs.append(output)
    
model.pool.register_forward_hook(layer1_hook)

In [7]:
input_ = torch.randn(1, 3, 32, 32)
output = model(input_)

In [8]:
outputs

[tensor([[[[6.5983e-01, 0.0000e+00, 8.7116e-01,  ..., 9.2938e-01,
            2.0758e-01, 8.2977e-01],
           [4.7891e-01, 5.2969e-01, 2.9371e-01,  ..., 0.0000e+00,
            4.4093e-01, 4.2059e-01],
           [1.4554e-01, 3.4143e-01, 5.5964e-01,  ..., 1.0561e+00,
            1.8811e-02, 2.4908e-01],
           ...,
           [9.3650e-01, 4.0230e-01, 0.0000e+00,  ..., 0.0000e+00,
            1.9329e-01, 2.8876e-01],
           [3.6867e-01, 8.4402e-01, 4.7803e-01,  ..., 2.1699e-01,
            6.2176e-01, 2.8500e-01],
           [5.5495e-02, 6.2154e-01, 9.3612e-02,  ..., 6.2921e-01,
            7.7591e-01, 4.9801e-01]],
 
          [[9.2797e-02, 2.6920e-01, 3.9068e-01,  ..., 2.9768e-01,
            3.1153e-01, 0.0000e+00],
           [8.5315e-01, 0.0000e+00, 2.8595e-01,  ..., 5.8789e-01,
            0.0000e+00, 1.3878e-01],
           [5.5357e-01, 9.9213e-01, 7.7977e-02,  ..., 6.8261e-01,
            6.9347e-01, 5.0794e-01],
           ...,
           [2.3878e-01, 7.7453e-01, 0.

In [67]:
print(outputs[0].shape)
print(outputs[1].shape)

torch.Size([1, 6, 14, 14])
torch.Size([1, 16, 5, 5])


## 加入自定義 initialization fuction

#### 對所有Conv2D層使用自定義initialization function
* weight : nn.init.kaiming_normal_
* bias : 全部輸入1

In [9]:
from torch.nn import init

def weights_init(m):
    #classname = m.__class__.__name__
    if isinstance(m, nn.Conv2d):
        #torch.nn.init.xavier_uniform_(m.weight)
        nn.init.kaiming_normal_(m.weight.data,
                                    a=0,
                                    mode='fan_out',
                                    nonlinearity='relu')
        if m.bias is not None:
            m.bias.data.fill_(1)
            
model.apply(weights_init)

TheModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

#### 查看 conv層的bias是否皆為1

In [10]:
for name, parameters in model.named_parameters():
    if ('conv' in name) and ('bias' in name):
        print(name, parameters)
        print('\n')

conv1.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1.], requires_grad=True)


conv2.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       requires_grad=True)


